In [1]:
# 95% CI from Monte Carlo
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.2e}'.format)

# Data import
file_path = "/Users/elchulito/Library/CloudStorage/OneDrive-polymtlus/0 - A_Database and methodology_PhD/PlasticFADE.xlsx"
sheet_name = "Uncertainty"
data_CI = pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:D", skiprows=1)
data_CI = data_CI.iloc[18:24] # Row index minus 3, change this range for other polymers
print(data_CI)

# Parameter estimates
x_i, tau_i, y_i, theta_i, z_i, eta_i = data_CI.iloc[:, 2].values
# Standard deviations of parameters
x_i_std, tau_i_std, y_i_std, theta_i_std, z_i_std, eta_i_std = data_CI.iloc[:, 3].values

             Process Parameter  Estimate Standard deviation
18  LDPE degradation       x_i  1.10e-02           5.75e-03
19               NaN     tau_i  5.00e-13           1.10e-01
20               NaN       y_i  1.04e-02           6.07e-03
21               NaN   theta_i  4.38e-03           5.03e-05
22               NaN       z_i  1.06e-04           4.47e-04
23               NaN     eta_i  3.24e-01           1.92e-01


In [3]:
# Input parameters
data_input = pd.read_excel(file_path, sheet_name="Results", usecols="A:D,F", skiprows=13)
data_input = data_input[data_input.iloc[:, 0] == "LDPE"]  # Change for different polymer types
print(data_input)
data_input.columns = ['Polymer', 'Compartment', 's', 'I_j', 'C_j']

# Monte Carlo setup
N = 10000
np.random.seed(42)
results = []

# Loop through each row of input
for index, row in data_input.iterrows():
    s = row['s']
    I_j = row['I_j']
    C_j = row['C_j']

    # Monte Carlo with log-normal distribution
    N = 10000
    x_i_samples = np.random.lognormal(np.log(x_i), x_i_std, N)
    tau_i_samples = np.random.lognormal(np.log(tau_i), tau_i_std, N)
    y_i_samples = np.random.lognormal(np.log(y_i), y_i_std, N)
    theta_i_samples = np.random.lognormal(np.log(theta_i), theta_i_std, N)
    z_i_samples = np.random.lognormal(np.log(z_i), z_i_std, N)
    eta_i_samples = np.random.lognormal(np.log(eta_i), eta_i_std, N)

    # Compute k_degr for each sample
    k_samples = x_i_samples * (s**tau_i_samples) * (y_i_samples * I_j**theta_i_samples + z_i_samples * C_j**eta_i_samples)
    k_samples = k_samples[np.isfinite(k_samples)]  # Filter invalid samples (good habit, especially when NaNs are found in the CIs)

    # 95% CI in log-space
    log_k = np.log10(k_samples)
    lower_bound = 10 ** np.percentile(log_k, 2.5)
    upper_bound = 10 ** np.percentile(log_k, 97.5)
    k_point = x_i * (s**tau_i) * (y_i * I_j**theta_i + z_i * C_j**eta_i)
    
    results.append({'Compartment': row['Compartment'], 'k_point': k_point, 'CI_lower': lower_bound, 'CI_upper': upper_bound})

# --- Display results ---
results_CI = pd.DataFrame(results)
print("\n", results_CI)
print(f"\n{N - len(k_samples)} out of {N} samples were invalid and removed.")

   Polymer (i) Compartment (j)  SA:V [cm-1]  I_j [W/m2]  C_j [CFU/ml]
35        LDPE             Air           25    1.00e+01      5.00e-01
36        LDPE         Topsoil           25    1.00e-01      6.70e+08
37        LDPE         Subsoil           25    0.00e+00      1.21e+08
38        LDPE           Beach           25    1.25e+01      1.25e+07
39        LDPE   Water surface           25    1.00e+01      2.50e+05
40        LDPE    Water column           25    0.00e+00      3.85e+04
41        LDPE        Sediment           25    0.00e+00      4.82e+05

      Compartment  k_point  CI_lower  CI_upper
0            Air 1.16e-04  1.14e-04  1.18e-04
1        Topsoil 9.52e-04  2.17e-04  1.82e-02
2        Subsoil 4.82e-04  7.13e-05  7.44e-03
3          Beach 3.46e-04  1.59e-04  2.58e-03
4  Water surface 1.80e-04  1.33e-04  5.18e-04
5   Water column 3.54e-05  1.24e-05  1.77e-04
6       Sediment 8.04e-05  2.10e-05  5.27e-04

0 out of 10000 samples were invalid and removed.


In [5]:
# Write confidence intervals back to Excel (without modifying)
import xlwings as xw

wb = xw.Book(file_path)  # file_path is your existing Excel file path
sheet = wb.sheets["Results"]

start_row = 50  # Change this index for other polymers
sheet.range(f'M{start_row}').options(index=False, header=False).value = results_CI['CI_lower'].values.reshape(-1, 1)
sheet.range(f'N{start_row}').options(index=False, header=False).value = results_CI['CI_upper'].values.reshape(-1, 1)

wb.save()
wb.close()